In [1]:
import numpy as np
import USGShydro #view via https://nbviewer.jupyter.org/github/DouglasPatton/Hydro/blob/master/helpers.ipynb

Loading BokehJS ...

to do:

USGShydro.py
<https://nbviewer.jupyter.org/github/DouglasPatton/Hydro/blob/master/helpers.ipynb>
- ~~download rainfall~~ 
- ~~download runoff~~
- ~~research time conversions~~
- ~~match data values aross time values~~
- ~~check for gaps in time
- ~~convert data to numpy array~~
- ~~create simple plot to view downloaded gage and precip data
- merge time series from different XML files capability?

RainfallRunoff.py
- create time series dataset of lags, etc.
- drop observations with missing values
- run rainfall-runoff model
- plot predicted runoff vs. actual
- predict days above flood stage with models trained over different time periods


-- create tool to query sites for availability of needed series
--compare sites and see if model relates to hydrologic featues, basin topography, landcover, etc.

In [2]:
site='sites=02314500' #Fargo, Ga below the Okefenokee NWR
start='startDT=2011-01-01T00:00%2b0000'#check start time formatting
end='endDT=2011-01-01T00:00-0400'
paramlist=['00045', '00065'] #must be entered as strings
modelfeatures=[] #not yet developed
try1=USGShydro.Hydrositedatamodel(site,start,end,paramlist,modelfeatures) #create the object

all series have matching times from start to end
all time steps are evenly spaced
2993 3


In [3]:
try1.simpleplot() #plot a time series of rainfall and gage height (above minimum for series)

In [4]:
try1.data_array[0:199:4,:]#print every 4th data point up to 200th observation

array([[0.0e+00, 0.0e+00, 7.0e-01],
       [1.0e+00, 0.0e+00, 7.0e-01],
       [2.0e+00, 0.0e+00, 7.0e-01],
       [3.0e+00, 0.0e+00, 7.0e-01],
       [4.0e+00, 0.0e+00, 7.1e-01],
       [5.0e+00, 0.0e+00, 7.1e-01],
       [6.0e+00, 0.0e+00, 7.0e-01],
       [7.0e+00, 0.0e+00, 7.0e-01],
       [8.0e+00, 0.0e+00, 7.1e-01],
       [9.0e+00, 0.0e+00, 7.1e-01],
       [1.0e+01, 0.0e+00, 7.0e-01],
       [1.1e+01, 0.0e+00, 7.0e-01],
       [1.2e+01, 0.0e+00, 7.0e-01],
       [1.3e+01, 0.0e+00, 7.0e-01],
       [1.4e+01, 0.0e+00, 7.0e-01],
       [1.5e+01, 0.0e+00, 7.0e-01],
       [1.6e+01, 0.0e+00, 7.1e-01],
       [1.7e+01, 0.0e+00, 7.1e-01],
       [1.8e+01, 0.0e+00, 7.2e-01],
       [1.9e+01, 0.0e+00, 7.1e-01],
       [2.0e+01, 0.0e+00, 7.1e-01],
       [2.1e+01, 2.0e-02, 7.2e-01],
       [2.2e+01, 3.0e-02, 7.4e-01],
       [2.3e+01, 2.0e-02, 7.6e-01],
       [2.4e+01, 1.0e-02, 7.6e-01],
       [2.5e+01, 1.0e-02, 7.6e-01],
       [2.6e+01, 0.0e+00, 7.6e-01],
       [2.7e+01, 0.0e+00, 7.

In [5]:
try1.requesturl

'https://waterservices.usgs.gov/nwis/iv/?format=waterml,2.0&sites=02314500&startDT=2011-01-01T00:00%2b0000&endDT=2011-02-01T00:00-0400&parameterCd=00045,00065&siteStatus=all'